# Gera Dados

Este é o arquivo responśavel por extrair e comparar os dados das planilhas `eleitores_estado_2018.csv` e `eleitores_estado_2021.csv`, colocando os resultados em `filiados_por_estado.xlsx`. As planilhas foram manualmente extraídas do site do TSE.

## Importação de bibliotecas

In [22]:
import pandas as pd

## Extraindo dados

Essas duas células apenas extraem os dados das planilhas e os colocam DataFrames, usados para manipulação desses dados. O conteúdo desses DataFrames também é checado. 

In [23]:
estados_2018 = pd.read_csv('eleitores_estado_2018.csv', sep=';')
estados_2021 = pd.read_csv('eleitores_estado_2021.csv', sep=';')
estados_2018.head()

,Abrangência,Partido,Eleitores,%
0,AC,AVANTE,664,"0,004"
1,AC,DC,1.250,"0,007"
2,AC,DEM,3.104,"0,018"
3,AC,MDB,8.728,"0,052"
4,AC,NOVO,22,"0,000"


In [24]:
estados_2021.head()

,Abrangência,Partido,Eleitores,%
0,AC,AVANTE,671,"0,004"
1,AC,CIDADANIA,1.333,"0,008"
2,AC,DC,1.124,"0,007"
3,AC,DEM,3.240,"0,020"
4,AC,MDB,9.023,"0,056"


Posteriormente, alguns nomes de partidos são trocados e padronizados entre as planilhas, a fim de facilitar a comparação dos dados entre elas, pois alguns partidos trocaram de nome entre os anos, ou sua sigla acabou sendo registrada de duas formas diferentes. Essas inconsistências infelizmente são comuns no banco de dados do TSE

In [25]:
estados_2018 = estados_2018.replace('PATRI', 'PATRIOTA')
estados_2018 = estados_2018.replace('PC DO B', 'PCDOB')
estados_2018 = estados_2018.replace('SD', 'SOLIDARIEDADE')
estados_2018 = estados_2018.replace('PR', 'PL')
estados_2018 = estados_2018.replace('PRB', 'REPUBLICANOS')
estados_2018 = estados_2018.replace('PPS', 'CIDADANIA')
estados_2018 = estados_2018.replace('PR', 'PL')

estados_2021 = estados_2021.replace('PATRI', 'PATRIOTA')
estados_2021 = estados_2021.replace('PC DO B', 'PCDOB')
estados_2021 = estados_2021.replace('SD', 'SOLIDARIEDADE')
estados_2021 = estados_2021.replace('PR', 'PL')
estados_2021 = estados_2021.replace('PRB', 'REPUBLICANOS')
estados_2021 = estados_2021.replace('PPS', 'CIDADANIA')
estados_2021 = estados_2021.replace('PR', 'PL')

estados_2018 = estados_2018.replace('ZZ', 'EXTERIOR')
estados_2021 = estados_2021.replace('ZZ', 'EXTERIOR')
estados_2018.head(10)

,Abrangência,Partido,Eleitores,%
0,AC,AVANTE,664,"0,004"
1,AC,DC,1.250,"0,007"
2,AC,DEM,3.104,"0,018"
3,AC,MDB,8.728,"0,052"
4,AC,NOVO,22,"0,000"
5,AC,PATRIOTA,270,"0,002"
6,AC,PCDOB,8.168,"0,049"
7,AC,PCB,84,"0,001"
8,AC,PCO,25,"0,000"
9,AC,PDT,2.207,"0,013"


## Juntando as planilhas

Posteriormente, cada linha das planilhas recebe um "id" único, composto por Abrangência (sigla do estado) e nome do partido (`Abr_Part`). Esses "id's" permitem que o método `.merge` do Pandas seja utilizado para juntar os dados das duas planilhas. 

In [26]:
# Criando os ids em estados_2018:
ids = list()
for row in estados_2018.index:
    ids.append(f"{estados_2018['Abrangência'][row]}_{estados_2018['Partido'][row]}")

estados_2018['id'] = ids
estados_2018.head(10)

,Abrangência,Partido,Eleitores,%,id
0,AC,AVANTE,664,"0,004",AC_AVANTE
1,AC,DC,1.250,"0,007",AC_DC
2,AC,DEM,3.104,"0,018",AC_DEM
3,AC,MDB,8.728,"0,052",AC_MDB
4,AC,NOVO,22,"0,000",AC_NOVO
5,AC,PATRIOTA,270,"0,002",AC_PATRIOTA
6,AC,PCDOB,8.168,"0,049",AC_PCDOB
7,AC,PCB,84,"0,001",AC_PCB
8,AC,PCO,25,"0,000",AC_PCO
9,AC,PDT,2.207,"0,013",AC_PDT


In [27]:
# Criando os ids em estados_2021:
ids = list()
for row in estados_2021.index:
    ids.append(f"{estados_2021['Abrangência'][row]}_{estados_2021['Partido'][row]}")

estados_2021['id'] = ids
estados_2021.head(10)

,Abrangência,Partido,Eleitores,%,id
0,AC,AVANTE,671,"0,004",AC_AVANTE
1,AC,CIDADANIA,1.333,"0,008",AC_CIDADANIA
2,AC,DC,1.124,"0,007",AC_DC
3,AC,DEM,3.240,"0,020",AC_DEM
4,AC,MDB,9.023,"0,056",AC_MDB
5,AC,NOVO,25,"0,000",AC_NOVO
6,AC,PATRIOTA,1.121,"0,007",AC_PATRIOTA
7,AC,PCB,72,"0,000",AC_PCB
8,AC,PCDOB,7.595,"0,047",AC_PCDOB
9,AC,PCO,35,"0,000",AC_PCO


In [28]:
# Juntando e checando a comparação das planilhas:
estados_cmp = pd.merge(estados_2021, estados_2018, on='id', how='outer', suffixes=('_2021', '_2018'))

estados_cmp['Eleitores_2018'] = estados_cmp['Eleitores_2018'].fillna(0)
estados_cmp['Eleitores_2021'] = estados_cmp['Eleitores_2021'].fillna(0)

estados_cmp.head(40)

,Abrangência_2021,Partido_2021,Eleitores_2021,%_2021,id,Abrangência_2018,Partido_2018,Eleitores_2018,%_2018
0,AC,AVANTE,671,"0,004",AC_AVANTE,AC,AVANTE,664,"0,004"
1,AC,CIDADANIA,1.333,"0,008",AC_CIDADANIA,AC,CIDADANIA,1.515,"0,009"
2,AC,DC,1.124,"0,007",AC_DC,AC,DC,1.250,"0,007"
3,AC,DEM,3.240,"0,020",AC_DEM,AC,DEM,3.104,"0,018"
4,AC,MDB,9.023,"0,056",AC_MDB,AC,MDB,8.728,"0,052"
5,AC,NOVO,25,"0,000",AC_NOVO,AC,NOVO,22,"0,000"
6,AC,PATRIOTA,1.121,"0,007",AC_PATRIOTA,AC,PATRIOTA,270,"0,002"
7,AC,PCB,72,"0,000",AC_PCB,AC,PCB,84,"0,001"
8,AC,PCDOB,7.595,"0,047",AC_PCDOB,AC,PCDOB,8.168,"0,049"
9,AC,PCO,35,"0,000",AC_PCO,AC,PCO,25,"0,000"


## Finalizando

Por fim, é criado um novo DataFrame, baseado no recente `estados_cmp`, que contém os dados de ambos os primeiros. Nesse, a coluna dos "id's" é retirada, uma vez que não é mais necessária, e dados redundantes de estados e partidos também são removidos. Também é adicionada uma nova coluna, que contém a diferença entre os dados de 2021 e 2018.

In [29]:
estados_final = pd.DataFrame()

for row in estados_cmp.index:
    if estados_cmp['id'][row].split("_")[0] != 'TOTAL':
        new_row = {
            'Abrangência': estados_cmp['id'][row].split("_")[0],
            'Partido': estados_cmp['id'][row].split("_")[1],
            'Filiados_2021': int(str(estados_cmp['Eleitores_2021'][row]).replace('.','')),
            'Filiados_2018': int(str(estados_cmp['Eleitores_2018'][row]).replace('.','')),
            'Diferença': int(str(estados_cmp['Eleitores_2021'][row]).replace('.','')) - int(str(estados_cmp['Eleitores_2018'][row]).replace('.',''))
        }
    estados_final = estados_final.append(new_row, ignore_index=True)
    
estados_final.head(10)

,Abrangência,Diferença,Filiados_2018,Filiados_2021,Partido
0,AC,7.0,664.0,671.0,AVANTE
1,AC,-182.0,1515.0,1333.0,CIDADANIA
2,AC,-126.0,1250.0,1124.0,DC
3,AC,136.0,3104.0,3240.0,DEM
4,AC,295.0,8728.0,9023.0,MDB
5,AC,3.0,22.0,25.0,NOVO
6,AC,851.0,270.0,1121.0,PATRIOTA
7,AC,-12.0,84.0,72.0,PCB
8,AC,-573.0,8168.0,7595.0,PCDOB
9,AC,10.0,25.0,35.0,PCO


Por fim, esse DataFrame é convertido em um arquivo do Excel (`.xlsx`) para análise dos dados. 

In [30]:
estados_final.to_excel('filiados_por_estado.xlsx')